### Using IPython Interactive Widgets to control a robot

In this notebook, you will learn how to use interactive **[widgets](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html)** to control a robot. We will first try to control a turtle from *[turtlesim](http://wiki.ros.org/action/fullsearch/turtlesim?action=fullsearch&context=180&value=linkto%3A%22turtlesim%22)* simulation.

In the turtlesim simulator we can guide a virtual turtle by sending him [_twist_](http://wiki.ros.org/geometry_msgs) messages. To simplify, everyone will run his own simulator, on his local roscore server.

In [ ]:
%env ROS_MASTER_URI=http://localhost:11311
import rospy
import roslaunch
import rosnode



We need also to start *roscore*, as we are working on our local computer

In [ ]:
import subprocess
roscore_process=subprocess.Popen(["roscore"])

We will start a turtlesim node using **roslaunch** command. This will be explained in more detail in next notebook.

In [ ]:
launch = roslaunch.scriptapi.ROSLaunch()
launch.start()
node2=roslaunch.core.Node("turtlesim", "turtlesim_node",name="turtlesim_node")
launch.launch(node2)


You should see a turtle swimming in the sea. I suggest that you rightlick on the turtlesim window bar and set "always on top". In this way you can see what is going on when you publish some messages. 

## Exercise:
Now use** *rosnode.rosnode_info* ** to see what it can do (its name is turtlesim_node).

In [ ]:
? rosnode.rosnode_info #You will receive some comments about the function

In [ ]:
rosnode.rosnode_info("turtlesim_node") #Put your code here

### Controling the turtle via a Twist command

With rosnode_info command You can see that there is plenty to do with the turtle.
To change its position we need to use **turtle1/cmd_vel** topic with a Twist type message


In [ ]:
from geometry_msgs.msg import Twist # we import the definition of a Twist message


We can check if the robot reacts to some twist message by setting up a publisher


In [ ]:
rospy.init_node("turtle_controller") #we initialize with your favorite node name

twist_publisher= rospy.Publisher("/turtle1/cmd_vel",Twist,queue_size=10) #we create a publisher for a
#"/turtle1/cmd_vel" topic

In [ ]:
### let's create some twist message
some_message=Twist()
some_message.angular.z=1
some_message.linear.x=10

twist_publisher.publish(some_message)

Try setting some different values and looking what the turtle does. 

While it gives us some intuition on how the robot behaves, we need to rerun the IPython cell to understand the behaviour. There is a better way :)

First let's define a function that modifies these 3 values in a message and then sends it



In [ ]:

def move_turtle(forward_vel=5,rotation_vel=5):
    '''A function to move turtle from turtlesim simulation
    
    Args:
        forward_vel (float): Linear velocity
        rotation_vel (float): Angular velocity'''
    message=Twist()
    message.angular.z=rotation_vel
    message.linear.x=forward_vel
    
    
    twist_publisher.publish(message)

We can now move the turtle using this function. 

In [ ]:
move_turtle(1,-1)

The fun part comes when we use widgets.

Let's put our function as an argument to a interact function from ipywidgets

In [ ]:
import ipywidgets


In [ ]:
ipywidgets.interact(move_turtle)

When you run the code above, you should get an two interactive sliders, that allow you to control the turtle. 
The interact function guessed the right controllers because we put a default values for parameters of our function

The interact function has many fun parameters to choose but in case of prototyping algorithms for robots we are mostly interested in couple of them


In [ ]:
ipywidgets.interact(move_turtle,
                    forward_vel=ipywidgets.FloatSlider(min=-10,max=10,step=2,value=0),
                   rotation_vel= ipywidgets.FloatSlider(min=-3,max=3,value=0))

In some situations we also want first to put some values and then to start the simulaiton. We can do this by using interact_manual

In [ ]:
ipywidgets.interact_manual(move_turtle,
                    forward_vel=ipywidgets.FloatSlider(min=-10,max=10,step=2,value=0),
                   rotation_vel= ipywidgets.FloatSlider(min=-3,max=3,value=0))

This is a more complicated use of interact. We have explicitly stated types of values (floats) by using *FloatSliders* and minimum and maximum values of these sliders. We have also used interact_manual which gives us additional button to "send" the values. Otherwise, it would be sending the values on every change bigger than the step value for a particular slider. In the case of many robot functions, we want first to set some "target" and then to publish this target. 

Instead of interact_manual you can also use continuous_update=False to run the simulation only when you release the mouse.

### Exercise:

We can teleport the turtle to some particular place using teleport_absolute service. Create widgets for x,y, theta to teleport turtle. Rememeber that the size of the field is 10x10 (x and y from 0 to 10) and theta is in radians (from 0 to 2 pi)


In [ ]:

from turtlesim.srv import TeleportAbsolute
from math import pi


## here create a service proxy function for /turtle1/teleport_absolute of TeleportAbsolute type
service_teleport = ...

def teleport_turtle(x=0.0,y=0.0,theta=0.0):
    service_teleport(x,y,theta)

# create an interact widget here with correct min max values for arguments
...

*HINT: You have an example of how to create a ServiceProxy function in previous notebook.*

### More widget types

There are more categories of widgets such as a color picker, button or select which could be used for example while configuring rosparam. What is cooler is that the type of widget can frequently be set automatically, by setting the default value of function argument.

In [ ]:
import rosparam

def set_Ono_occupation(occupation=["programmer","ROS tutor","model"]):
    rosparam.set_param("/Ono/occupation",occupation)
    print("Ono occupation changed to {occupation}".format(occupation=occupation))
ipywidgets.interact(set_Ono_occupation)

The same can be done with *decorators*, that is, you add @ipywidgets.interact at the top of your function's definition.

In [ ]:
import rosparam
@ipywidgets.interact #we use a 
def set_Ono_occupation(occupation=["programmer","ROS tutor","model"]):
    print("Ono occupation changed to {occupation}".format(occupation=occupation))
    rosparam.set_param("/Ono/occupation",occupation)
    


### Close roscore

In [ ]:


roscore_process.kill()

### WRAPUP

What you have learned:

1. You have used ipython interactive widgets to control a turtlesim.
2. You have learned different options of this command to modify it to needs.
3. You used the widget as a decorator.



## Nice job! 

Now, you can continue our journey with next notebook **[Visualizing_robot_senses.ipynb](Visualizing_robot_senses.ipynb)**.